## 강의 목표 :
- 데이터 로딩
- 텍스트 데이터 Feature 처리
- 데이터 학습 및 예측
- Kaggle 제출

## 추가 조사 사항
- SGDCalssifier 란 무엇인지 설명 필요??? 
> - http://dreamtamer.egloos.com/7321983
> - (SVM) http://blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220969601609&parentCategoryNo=&categoryNo=87&viewDate=&isShowPopularPosts=false&from=postView
> - (확률적경사하강법) http://blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220969826890&redirect=Dlog&widgetTypeCall=true

In [2]:
import pandas as pd

## Load Data

In [6]:
train = pd.read_csv('data/train.tsv',sep='\t',index_col='PhraseId')

In [11]:
print(train.shape)
train.head()

(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2


In [8]:
test = pd.read_csv('data/test.tsv',sep='\t',index_col='PhraseId')

In [10]:
print(test.shape)
test.head()

(66292, 2)


,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine


## Vectorize Phrase

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=100,ngram_range=(1,2))
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [27]:
vectorizer.fit(train['Phrase'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [41]:
X_train = vectorizer.transform(train['Phrase'])
X_train

<156060x100 sparse matrix of type '<class 'numpy.int64'>'
	with 399930 stored elements in Compressed Sparse Row format>

In [43]:
X_test = vectorizer.transform(test['Phrase'])
X_test

<66292x100 sparse matrix of type '<class 'numpy.int64'>'
	with 157536 stored elements in Compressed Sparse Row format>

In [31]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [40]:
columns = vectorizer.get_feature_names()

pd.DataFrame(X_train[:100].toarray(),columns=columns).head()

,about,action,all,an,and,and the,any,are,as,at,...,way,well,what,which,who,will,with,work,you,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
y_train = train['Sentiment']

y_train.head()

PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

## 모델 학습 및 예측

In [18]:
from sklearn.linear_model import SGDClassifier

# help(SGDClassifier)

model = SGDClassifier(random_state=37)
model

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [64]:
help(SGDClassifier)

Help on class SGDClassifier in module sklearn.linear_model.stochastic_gradient:

class SGDClassifier(BaseSGDClassifier)
 |  Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
 |  
 |  This estimator implements regularized linear models with stochastic
 |  gradient descent (SGD) learning: the gradient of the loss is estimated
 |  each sample at a time and the model is updated along the way with a
 |  decreasing strength schedule (aka learning rate). SGD allows minibatch
 |  (online/out-of-core) learning, see the partial_fit method.
 |  For best results using the default learning rate schedule, the data should
 |  have zero mean and unit variance.
 |  
 |  This implementation works with data represented as dense or sparse arrays
 |  of floating point values for the features. The model it fits can be
 |  controlled with the loss parameter; by default, it fits a linear support
 |  vector machine (SVM).
 |  
 |  The regularizer is a penalty added to the loss function tha

In [48]:
model.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [56]:
predictions = model.predict(X_test)

## Submit

In [60]:
submission = pd.read_csv('data/sampleSubmission.csv',index_col='PhraseId')

submission['Sentiment'] = predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [61]:
submission.to_csv('prediction_result1.csv')